# Baseline Evaluation

This notebook evaluates baseline retrieval methods (BM25 and TF-IDF) on TREC-COVID and Climate-Fever datasets.

## How to Run

1. Press "Run All" to execute all cells
2. All required files will be created automatically if missing
3. Metrics and plots will be saved to `data/evaluate/baseline/`


In [1]:
# Setup: imports and path configuration
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json

# Notebook is located at notebook/baseline.ipynb
project_root = Path.cwd().parents[0]

# Define core configuration
datasets = ["trec_covid", "climate_fever"]
retrieval_methods = ["bm25", "tfidf"]  # baseline retrieval methods we actually support
data_base = project_root / "data"

sys.path.insert(0, str(project_root / 'src'))

from evaluate import (
    compute_metrics_from_files,
    save_metrics_to_csv,
)
from evaluate.utils import (
    ensure_directory,
    create_summary_table,
)
from notebook.run_api import ensure_runs

print("Setup complete! Project root:", project_root)

Setup complete! Project root: c:\Users\Lenovo\CodeProjects\VSC\domain-specific-query-expansion-with-llms


c:\Users\Lenovo\CodeProjects\VSC\domain-specific-query-expansion-with-llms\.venv\Lib\site-packages\beir\datasets\data_loader.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Ensure ingest outputs and baseline runs

Use the programmatic ingest + retrieval API to materialize real artifacts under data/ingest/{dataset} and data/retrieve/baseline/ if they are missing.


In [2]:
# Ensure output directory exists
ensure_directory(data_base / "retrieve" / "baseline")
ensure_directory(data_base / "evaluate" / "baseline")

# Ensure baseline runs exist
runs = ensure_runs(method="baseline", datasets=datasets, retrieval_methods=retrieval_methods, top_k=100)
print("Baseline runs ensured:\n", json.dumps(runs, indent=2, default=str))

=== Dataset: trec_covid ===
[trec_covid] Ingested artifacts are stale (newer extract found); will re-ingest.
[trec_covid] Extracted dataset is stale (newer download found); will re-extract.
[trec_covid] Re-extracting from existing download...
Downloaded and renamed to C:\Users\Lenovo\CodeProjects\VSC\domain-specific-query-expansion-with-llms\data\download\trec_covid.zip
Extracting C:\Users\Lenovo\CodeProjects\VSC\domain-specific-query-expansion-with-llms\data\download\trec_covid.zip to C:\Users\Lenovo\CodeProjects\VSC\domain-specific-query-expansion-with-llms\data\extract\trec_covid ...
Finished extracting to C:\Users\Lenovo\CodeProjects\VSC\domain-specific-query-expansion-with-llms\data\extract\trec_covid
Extracted trec_covid
[trec_covid] Ingesting dataset...


  0%|          | 0/171332 [00:00<?, ?it/s]

Ingested trec_covid


KeyboardInterrupt: 

## Load and display dataset statistics

In [ ]:
# Load and display dataset statistics from manifest.json
print("\n=== Dataset Statistics ===")
from ingest.core import get_ingested_dataset_paths

dataset_stats = {}
for dataset in datasets:
    paths = get_ingested_dataset_paths(dataset)
    if paths.manifest.exists():
        manifest_data = json.loads(paths.manifest.read_text(encoding="utf-8"))
        dataset_stats[dataset] = {
            "doc_count": manifest_data.get("doc_count", "N/A"),
            "query_count": manifest_data.get("query_count", "N/A"),
            "qrels_count": manifest_data.get("qrels_count", "N/A"),
            "vocab_size": manifest_data.get("vocab_size", "N/A"),
            "split": manifest_data.get("split", "N/A"),
        }
        print(f"{dataset}:")
        print(f"  Documents: {dataset_stats[dataset]['doc_count']}")
        print(f"  Queries: {dataset_stats[dataset]['query_count']}")
        print(f"  Qrels: {dataset_stats[dataset]['qrels_count']}")
        print(f"  Vocab size: {dataset_stats[dataset]['vocab_size']}")
        print(f"  Split: {dataset_stats[dataset]['split']}")

# Save dataset statistics to data/evaluate
if dataset_stats:
    stats_path = data_base / "evaluate" / "dataset_stats.json"
    ensure_directory(stats_path.parent)
    with open(stats_path, 'w', encoding='utf-8') as f:
        json.dump(dataset_stats, f, indent=2)
    print(f"\nDataset statistics saved to: {stats_path}")

## Compute Metrics for All 4 Combos

Compute nDCG@10, MAP, Recall@100, MRR for:
- BM25 × TREC-COVID
- BM25 × Climate-Fever  
- TF-IDF × TREC-COVID
- TF-IDF × Climate-Fever


In [ ]:
# Compute metrics for all combinations using real ingest outputs
all_metrics = {}

for dataset in datasets:
    qrels_path = data_base / "ingest" / dataset / "qrels.csv"

    for retrieval in retrieval_methods:
        run_path = data_base / "retrieve" / "baseline" / f"{dataset}_{retrieval}.csv"
        metric_path = data_base / "evaluate" / "baseline" / f"{dataset}_{retrieval}.csv"

        # Compute and save metrics
        metrics = compute_metrics_from_files(str(run_path), str(qrels_path), k=10)
        save_metrics_to_csv(
            metrics,
            str(metric_path),
            dataset=dataset,
            method="baseline",
            retrieval=retrieval,
        )

        all_metrics[(dataset, "baseline", retrieval)] = metrics

print("\nMetrics computation complete!")


## Summary Table


In [ ]:
# Create and display summary table
summary_df = create_summary_table(all_metrics)
summary_path = data_base / "evaluate" / "baseline" / "summary.csv"
summary_df.to_csv(summary_path, index=False)

print("Summary Metrics Table:")
print(summary_df.to_string(index=False))
print(f"\nSaved to: {summary_path}")


## Plot nDCG@10 Comparison


In [ ]:
# Plot nDCG@10 comparison
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(datasets))
width = 0.35

bm25_scores = [all_metrics[(d, "baseline", "bm25")]["ndcg@10"] for d in datasets]
tfidf_scores = [all_metrics[(d, "baseline", "tfidf")]["ndcg@10"] for d in datasets]

bars1 = ax.bar(x - width/2, bm25_scores, width, label="BM25", alpha=0.8)
bars2 = ax.bar(x + width/2, tfidf_scores, width, label="TF-IDF", alpha=0.8)

ax.set_xlabel("Dataset", fontsize=12)
ax.set_ylabel("nDCG@10", fontsize=12)
ax.set_title("Baseline nDCG@10 (BM25 vs TF-IDF)", fontsize=14, fontweight="bold")
ax.set_xticks(x)
ax.set_xticklabels(datasets)
ax.legend()
ax.grid(axis="y", alpha=0.3)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2.0, height,
                f"{height:.3f}", ha="center", va="bottom", fontsize=9)

plt.tight_layout()

# Save plot
plot_path = data_base / "evaluate" / "baseline" / "ndcg.png"
plt.savefig(plot_path, dpi=150, bbox_inches="tight")
print(f"Plot saved to: {plot_path}")
plt.show()


## Robustness Analysis: Compute Query Slices


In [ ]:
# Compute query slices for each dataset using ingest outputs (queries + vocab)
from evaluate.robustness_slices import compute_query_slices, load_vocabulary

for dataset in datasets:
    vocab_path = data_base / "ingest" / dataset / "vocab_top50k.txt"
    queries_path = data_base / "ingest" / dataset / "queries.csv"
    slices_path = data_base / "evaluate" / f"{dataset}.csv"

    # Load vocabulary
    vocab = load_vocabulary(str(vocab_path), top_n=50000)

    # Load queries.csv (query_id,text,split)
    queries_df = pd.read_csv(queries_path)
    # Convert query_id to string to match run file format
    queries = {str(row["query_id"]): row["text"] for _, row in queries_df.iterrows()}

    for retrieval in retrieval_methods:
        run_path = data_base / "retrieve" / "baseline" / f"{dataset}_{retrieval}.csv"
        # Compute slices
        slices = compute_query_slices(
            queries,
            str(run_path),
            vocab=vocab,
            output_file=str(slices_path),
        )

    familiar = sum(1 for s in slices.values() if s["label"] == "familiar")
    unfamiliar = len(slices) - familiar

    print(f"{dataset}: {familiar} familiar, {unfamiliar} unfamiliar queries")
    print(f"Slices saved to: {slices_path}")

print("\nRobustness analysis complete!")
